In [1]:
import numpy as np
import pandas as pd
import os
import re
import pathlib

import src.SimData  as sim_data

In [2]:
###* Experimental Data CO 2

schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "fracCO2ini_relO2": "frac_CO2",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "Tgas":         "Tgas",
    "O mean":       "O",
    "CO mean":      "CO",
    "MeanLossProb": "gamma_exp"
}

file_path = "Experimental_data_CO/Data_CO2_TD.xlsx"

pd_exp = pd.read_excel(file_path)
pd_result_exp = pd.DataFrame()

for old, new in schema.items():
    if old in pd_exp.columns:
        pd_result_exp[new] = pd_exp[old]


pd_result_exp["Tw"] = pd_result_exp["Tw"] + 273.15 
# pd_result_exp["current"] = pd_result_exp["current"] * 1e-3
pd_result_exp["Tnw"] = pd_result_exp["Tw"] + 0.45 * (pd_result_exp["Tgas"] - pd_result_exp["Tw"])
pd_result_exp["frac_O2"] = 1.0 - pd_result_exp["frac_CO2"]

print(pd_result_exp.head())
print(pd_result_exp.shape)

   pressure  current  frac_CO2      Tw        Tgas  gamma_exp         Tnw  \
0       0.4       40       0.0  323.15  360.560032   0.001819  339.984514   
1       0.6       40       0.0  323.15  372.733684   0.000995  345.462658   
2       0.8       40       0.0  323.15  384.630566   0.000841  350.816255   
3       1.0       40       0.0  323.15  396.252108   0.000775  356.045949   
4       1.5       40       0.0  323.15  424.110735   0.000827  368.582331   

   frac_O2  
0      1.0  
1      1.0  
2      1.0  
3      1.0  
4      1.0  
(332, 8)


In [3]:

folder_path = "Experimental_data_CO/ResultsLoKI"
file_name_list = os.listdir(folder_path)
nb_list = sorted([int(file.split('_')[1]) for file in file_name_list])

def _read_output_densities(output_densities_path, skiprows=5):
        
    parts = []
    for line in output_densities_path.read_text().splitlines()[skiprows:]:
        toks = [tok for tok in line.split() if tok]
        if len(toks) < 2:
            continue
        elif toks[0] == '|':
            continue
            
        parts.append(toks)
        
    max_len = max(len(p) for p in parts)
    full_rows = [p for p in parts if len(p) == max_len]
        
    arr = np.array(full_rows, dtype=object)
    names = arr[:, 0].tolist()
    values = arr[:, 1].astype(float)

    ##! Fixed for now
    O2_index = [0, 1, 2, 3, 4]
    O_index = [5, 6, 7, 8]
    O3_index = [9, 10]
    CO2_index = [11, 12, 13]
    CO_index = [14, 15, 16]
    C_index = [17]
    
    O2_den = np.sum(values[O2_index])
    O_den = np.sum(values[O_index])
    O3_den = np.sum(values[O3_index])
    CO2_den = np.sum(values[CO2_index])
    CO_den = np.sum(values[CO_index])
    C_den = np.sum(values[C_index])
    
    vec_den = [O2_den, O_den, O3_den, CO2_den, CO_den, C_den]
    names = ['O2', 'O', 'O3', 'CO2', 'CO', 'C']
    return vec_den, names


data_LoKI = np.zeros((pd_result_exp.shape[0],6), dtype=float)

for counter in range(pd_result_exp.shape[0]):
    if counter in nb_list:
        
        try: 
            file_name = f"Experimental_data_CO/ResultsLoKI/sim2_{counter}/chemFinalDensities.txt"
            out_path = pathlib.Path(file_name)
            vec_den, names = _read_output_densities(out_path)
        except:
            vec_den = [0] * len(vec_den)
            
        for i in range(len(vec_den)):
            data_LoKI[counter, i] = vec_den[i]
        
    else:
        pass



In [4]:


print(pd_result_exp.columns)
print(names)


for idx, name in enumerate(names):
    pd_result_exp[name] = data_LoKI[:, idx] * 1e-6
    # pd_result_exp[name] = data_LoKI[:, idx]


mask = (pd_result_exp[names] != 0).all(axis=1)
pd_result_clean_exp = pd_result_exp.loc[mask].reset_index(drop=True)

pd_result_clean_exp['Tw'] = pd_result_clean_exp['Tw'] - 273.15

path_excel = "Experimental_data_CO/Data_CO2_LoKI_TD.xlsx"
pd_result_clean_exp.to_excel(path_excel)

print(pd_result_clean_exp.head())




Index(['pressure', 'current', 'frac_CO2', 'Tw', 'Tgas', 'gamma_exp', 'Tnw',
       'frac_O2'],
      dtype='object')
['O2', 'O', 'O3', 'CO2', 'CO', 'C']
   pressure  current  frac_CO2    Tw        Tgas  gamma_exp         Tnw  \
0       1.5       40       1.0  50.0  555.183313   0.000317  427.564991   
1       2.0       40       1.0  50.0  606.771416   0.000274  450.779637   
2       3.0       40       1.0  50.0  680.737954   0.000297  484.064579   
3       5.0       40       1.0  50.0  757.395050   0.000317  518.560272   
4       0.8       20       1.0  50.0  419.735699   0.000175  366.613565   

   frac_O2            O2             O            O3           CO2  \
0      0.0  6.075358e+13  5.773550e+13  1.388414e+09  3.776473e+13   
1      0.0  7.875792e+13  8.033113e+13  1.634777e+09  4.941946e+13   
2      0.0  1.269342e+14  1.010707e+14  2.873421e+09  8.756550e+13   
3      0.0  2.204738e+14  1.425030e+14  5.013036e+09  1.648490e+14   
4      0.0  3.316187e+13  2.928601e+13  7.2510

In [5]:

### store in buffer data in hdf5 style

schema_buffer = {
    "pressure":     "pressure",
    "current":      "current",
    "frac_CO2":     "frac_CO2",
    "Tgas":         "Tgas",
    "Tnw":          "Tnw", 
    "Tw":           "Tw",
    "O":            "O",
    "O2":           "O2",
    "O3":           "O3",
    "C":            "C",
    "CO":           "CO",
    "CO2":          "CO2",
    "gamma_exp":    "gamma_exp"
}


output_file = "Experimental_data_CO_LoKI_TD.hdf5"
file_path = ["Experimental_data_CO/Data_CO2_LoKI_TD.xlsx"]


data_loader = sim_data.DataLoader(schema_buffer, output_file=output_file, files_path=file_path)
data_loader.load_data(force_update=True)

